<a href="https://colab.research.google.com/github/Nilufayeasmin299/Phishig-detection-using-both-machine-learning-and-deep-learning-techniques/blob/main/LSTM_phishing_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
#loading dataset
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/Second paper/Dataset/URL-Dataset.csv')
df.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,0
1,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,0
2,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,0
3,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,0
4,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1


In [9]:
df.columns

Index(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report',
       'Result'],
      dtype='object')

In [10]:
df.shape

(11055, 31)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   having_IP_Address            11055 non-null  int64
 1   URL_Length                   11055 non-null  int64
 2   Shortining_Service           11055 non-null  int64
 3   having_At_Symbol             11055 non-null  int64
 4   double_slash_redirecting     11055 non-null  int64
 5   Prefix_Suffix                11055 non-null  int64
 6   having_Sub_Domain            11055 non-null  int64
 7   SSLfinal_State               11055 non-null  int64
 8   Domain_registeration_length  11055 non-null  int64
 9   Favicon                      11055 non-null  int64
 10  port                         11055 non-null  int64
 11  HTTPS_token                  11055 non-null  int64
 12  Request_URL                  11055 non-null  int64
 13  URL_of_Anchor                11055 non-null  i

In [12]:
# shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
data = df.sample(frac=1).reset_index(drop=True)
data.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,-1,1,1,1,-1,-1,1,1,1,1,-1,-1,1,1,-1,1,1,0,1,1,1,1,1,1,1,-1,1,1,1,1
1,1,-1,1,1,1,-1,1,-1,1,1,1,1,-1,-1,0,-1,1,1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,0
2,1,-1,1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,1,1,1,1,-1,1,0,1,1,0,1,1
3,1,-1,1,-1,1,-1,0,0,-1,-1,-1,1,-1,-1,0,-1,-1,1,0,1,1,-1,1,1,1,-1,-1,-1,0,1,0
4,-1,1,-1,-1,-1,-1,1,0,-1,1,1,-1,1,-1,1,-1,1,-1,1,1,1,1,1,1,-1,0,1,1,1,1,0


In [13]:
# Sepratating & assigning features and target columns to X & y
y = data['Result']
X = data.drop('Result',axis=1)
X.shape, y.shape

((11055, 30), (11055,))

In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_=scaler.fit_transform(X)

In [15]:
y_=y.to_numpy()

In [16]:
X_=X_.reshape(X_.shape[0],X_.shape[1],1)

In [17]:
X_.shape

(11055, 30, 1)

In [18]:
y_.shape

(11055,)

In [19]:
import numpy
from numpy import mean
from numpy import std
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Dropout
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold
from keras.layers import Flatten, Activation, LSTM
import time

In [20]:
import tensorflow as tf
import random 

In [21]:
import random
random.seed( 1 )
tf.random.set_seed(2)

In [22]:
# lists to collect the performance metrics during each fold of the n-fold cross validation
accuracy=[]
loss=[]
recall=[]
f1=[]
precision=[]
Time=[]
start = time.time()

In [23]:
epochs = 10
batch_size=100
num_folds=10
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X_, y_):

  # Define the model architecture
  model = Sequential()
  model.add(LSTM(256, activation='relu', input_shape=(X_[0].shape)))
  model.add(Dropout(0.7))
  model.add(Dense(1, activation='sigmoid'))


  # Compile the model
  model.compile(loss='binary_crossentropy', 
                optimizer='adam',
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(X_[train], y_[train],
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)

  # Generate generalization metrics
  y_pred = model.predict_classes(X_[test])
  scores = model.evaluate(X_[test], y_[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  accuracy.append(scores[1] * 100)
  loss.append(scores[0])
  recall.append(recall_score(y_[test], y_pred))
  f1.append(f1_score(y_[test], y_pred))
  precision.append(precision_score(y_[test], y_pred))

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
100/100 [==============================] - 37s 153ms/step - loss: 0.6588 - accuracy: 0.6224
Epoch 2/10
100/100 [==============================] - 15s 153ms/step - loss: 0.6084 - accuracy: 0.6600
Epoch 3/10
100/100 [==============================] - 15s 151ms/step - loss: 0.6039 - accuracy: 0.6364
Epoch 4/10
100/100 [==============================] - 15s 153ms/step - loss: 0.5583 - accuracy: 0.7009
Epoch 5/10
100/100 [==============================] - 15s 152ms/step - loss: 0.4687 - accuracy: 0.7826
Epoch 6/10
100/100 [==============================] - 15s 152ms/step - loss: 0.5262 - accuracy: 0.7497
Epoch 7/10
100/100 [==============================] - 15s 153ms/step - loss: 0.3748 - accuracy: 0.8385
Epoch 8/10
100/100 [==============================] - 15s 154ms/step - loss: 0.3153 - accuracy: 0.8688
Epoch 9/10
100/100 [==============================] - 15s 154ms/step - loss: 0.

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Score for fold 1: loss of 0.2639980912208557; accuracy of 89.51175212860107%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/10
100/100 [==============================] - 17s 153ms/step - loss: 0.6313 - accuracy: 0.6288
Epoch 2/10
100/100 [==============================] - 15s 154ms/step - loss: 0.5478 - accuracy: 0.7099
Epoch 3/10
100/100 [==============================] - 15s 152ms/step - loss: 0.4705 - accuracy: 0.7674
Epoch 4/10
100/100 [==============================] - 15s 152ms/step - loss: 0.3899 - accuracy: 0.8483
Epoch 5/10
100/100 [==============================] - 15s 152ms/step - loss: 0.2951 - accuracy: 0.8843
Epoch 6/10
100/100 [==============================] - 15s 153ms/step - loss: 0.2870 - accuracy: 0.8880
Epoch 7/10
100/100 [==============================] - 15s 154ms/step - loss: 0.2797 - accuracy: 0.8900
Epoch 8/10
100/100 [==============================] - 15s 152ms/step - loss: 0.2745 - accuracy: 0.8893
Ep

In [24]:
# obtain approximate training time
end_time = time.time()
time = (end_time - start)     #obtain time taken to train the model

In [25]:
# summarize the distribution of scores
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean(accuracy), std(accuracy)))                   #mean of all the accuracies fom the n folds and standard deviation
print('Precision Mean: %.3f, Standard Deviation: %.3f' % (mean(precision), std(precision)))          #mean of all the precisions fom the n folds and standard deviation
print('Recall Mean: %.3f, Standard Deviation: %.3f' % (mean(recall), std(recall)))
print('F1 Mean: %.3f, Standard Deviation: %.3f' % (mean(f1), std(f1)))
print('Total time Mean: %.3f' % (time/10))
print('loss Mean: %.3f, Standard Deviation: %.3f' % (mean(loss), std(loss)))

Accuracy Mean: 90.384, Standard Deviation: 0.881
Precision Mean: 0.910, Standard Deviation: 0.021
Recall Mean: 0.918, Standard Deviation: 0.021
F1 Mean: 0.914, Standard Deviation: 0.009
Total time Mean: 431.779
loss Mean: 0.250, Standard Deviation: 0.024
